In [2]:
import tweepy
import os
import pandas as pd
import yfinance as yf
import openai
import big_baller_moves
import json
import requests
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
from urllib3.exceptions import NewConnectionError  # New import (CHANGED)

from bs4 import BeautifulSoup

In [3]:
#Put your Bearer Token in the parenthesis below
client1 = tweepy.Client(bearer_token=big_baller_moves.bossman_tingz["twitter_api_key"])

# Authenticate
auth = tweepy.OAuth1UserHandler(big_baller_moves.bossman_tingz["consumer_key"], big_baller_moves.bossman_tingz["consumer_secret"], big_baller_moves.bossman_tingz["access_token"], 
                                big_baller_moves.bossman_tingz["access_token_secret"])

# Create API object
api = tweepy.API(auth)

# openai key
openai.api_key = big_baller_moves.bossman_tingz["openai_api_key"]

In [4]:
#enter username to get the user ID
username = "Heisenberg_100k"

try:
    user = client1.get_user(username=username)
    user_id = user_id = user.data.id
    print("User ID for", username, ":", user_id)
except Exception as e:
    print("Error:", e)

User ID for Heisenberg_100k : 1750349614786691072


In [6]:
#get all tweets (paginator)
paginator = tweepy.Paginator(
    client1.get_users_tweets,               
    user_id,                                   
    exclude=['retweets', 'replies'],     
    max_results=100, 
    tweet_fields=['id', 'text', 'created_at', 'entities', 'attachments'],  
    media_fields=['preview_image_url', 'url'],
    expansions=['attachments.media_keys', 'author_id'],
    limit=2                           
)


listy = []
for page in paginator:
    listy.extend(page.data)

df = pd.DataFrame(listy)
df

,attachments,author_id,created_at,edit_history_tweet_ids,entities,id,text
0,NaN,1750349614786691072,2024-05-29 16:06:26+00:00,[1795849187344699468],NaN,1795849187344699468,RIP.\n\nNeed a miracle to breakeven on this one.
1,{'media_keys': ['3_1795812167582904320']},1750349614786691072,2024-05-29 13:39:20+00:00,[1795812171722723428],"{'cashtags': [{'start': 0, 'end': 4, 'tag': 'V...",1795812171722723428,"$VIX Closed half, 25 contracts\n\nIn 2.98 (May..."
2,NaN,1750349614786691072,2024-05-28 20:40:01+00:00,[1795555649134215214],"{'cashtags': [{'start': 0, 'end': 4, 'tag': 'I...",1795555649134215214,$IWM well today’s entry was not ideal.\n\nBut ...
3,{'media_keys': ['3_1795472177233604608']},1750349614786691072,2024-05-28 15:08:20+00:00,[1795472181117551039],"{'cashtags': [{'start': 0, 'end': 4, 'tag': 'I...",1795472181117551039,$IWM Open\n\nJust a small $1.9k bet for this F...
4,NaN,1750349614786691072,2024-05-27 19:08:21+00:00,[1795170192571384017],"{'cashtags': [{'start': 0, 'end': 4, 'tag': 'I...",1795170192571384017,$IWM Watchlist.\n\nIf inflation PCE #’s come i...
...,...,...,...,...,...,...,...
190,NaN,1750349614786691072,2024-04-12 04:48:26+00:00,[1778646333093818441],NaN,1778646333093818441,💰
191,NaN,1750349614786691072,2024-04-11 20:02:37+00:00,[1778514009383473156],NaN,1778514009383473156,Every trade today was a good one.\n\nMore tomo...
192,NaN,1750349614786691072,2024-04-11 18:17:36+00:00,[1778487580121862486],"{'annotations': [{'start': 1, 'end': 4, 'proba...",1778487580121862486,$CLSK\n\nWhoa.
193,{'media_keys': ['3_1778482031351246848']},1750349614786691072,2024-04-11 17:55:34+00:00,[1778482034832519277],"{'annotations': [{'start': 1, 'end': 4, 'proba...",1778482034832519277,$CLSK yooooo it’s going back up!\n\nSold cc’s ...


In [7]:
#extract urls for tweets with images
def get_display_url(entities):
    if isinstance(entities, dict) and "urls" in entities:
        urls = entities["urls"]
        display_urls = [url.get("display_url") for url in urls]
        return display_urls
    else:
        return 0

# Apply the function to make the display_urls a column in our dataframe
df["image_url"] = df["entities"].apply(get_display_url)

selected_columns = ['id', 'text', 'created_at', 'image_url']

# Creating a new DataFrame with selected columns
heisenberg_tweets = df[selected_columns].copy()

In [8]:
client = openai
pd.set_option('display.max_colwidth', None)

In [9]:
#old message
system_msg = "Tell me if the message is suggesting opening a long, short or N/A and explain why. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short."

In [10]:
#translates images to option play 
def get_response_image(text) :
    if text == 0:
        return "No image available"
    if not isinstance(text, str):
        print(text)
        text = text[0]
    try:
        response = client.chat.completions.create(model="gpt-4o", messages=[
                            {
                              "role": "user",
                              "content": [
                                {"type": "text", "text": "What kind of stock purchase is this image describing? If it is an option play like a call or a put please specify."},
                                {
                                  "type": "image_url",
                                  "image_url": {
                                    "url": text,
                                  },
                                },
                              ],
                            }
                          ], max_tokens=300,)
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        pass

In [11]:
def dynamic_prompting(text, sys_prompt, user_prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": sys_prompt
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": user_prompt},
                        {"type": "text", "text": text}
                    ]
                }
            ],
            max_tokens=300,
        )
        
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        pass


In [ ]:
def initialize_webdriver():
    options = Options()
    options.add_argument("--headless")  # Run in background without opening a browser window
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-popup-blocking")
    options.add_argument("--disable-dev-tools")
    options.add_argument('blink-settings=imagesEnabled=false')

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [22]:
def get_jpg_url(driver, link):
    max_attempts = 3
    initial_wait = 0.5
    max_wait = 10

    for attempt in range(max_attempts):
        url = None
        try:
            if isinstance(link, list):  # Extract first element if URL is a list
                for u in link:
                    if "twitter" in u:
                        url = u
                        break
            elif not link:
                print("No Link Provided")
                return None
            else:
                url = link

            if not url:
                return None

            url = f'https://{url}'
            print(f"Attempting to access URL: {url}")

            driver.get(url)
            # Wait for potential redirects and the page to stabilize
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'article')))
            time.sleep(1)  # Extended sleep to ensure the page is loaded

            images = driver.find_elements(By.TAG_NAME, "img")
            for img in images:
                img_src = img.get_attribute('src')
                if 'media' in img_src:
                    print("img_src: " + img_src)
                    return img_src

            print("No media images found on the page")
            return None

        except Exception as e:
            print(f"General error processing URL {url}: {e}")
            time.sleep(60)

    print("Maximum attempts reached, aborting")
    return None

In [26]:
driver = initialize_webdriver()
for i in range(len(heisenberg_tweets)):
    try:
        heisenberg_tweets.loc[i, 'jpg_url'] = get_jpg_url(driver, heisenberg_tweets.loc[i, 'image_url'])
        if i % 80 == 0 and i != 0:
            driver.quit()
            print("sleepy time")
            time.sleep(90)
            driver = initialize_webdriver()
        if i > 0 :
            driver.close()
    except Exception as e:
        print(f"Error processing row {i}: {e}")

driver.quit()

No Link Provided
Attempting to access URL: https://pic.twitter.com/MTwtai7VTt
img_src: https://pbs.twimg.com/media/GOwBptwacAAuYLR?format=jpg&name=small
No Link Provided
Attempting to access URL: https://pic.twitter.com/szyGLaP9Fl
img_src: https://pbs.twimg.com/media/GOrMbrAbMAA-o5k?format=jpg&name=small
No Link Provided
No Link Provided
Attempting to access URL: https://pic.twitter.com/dslHjMOuNR
img_src: https://pbs.twimg.com/media/GOR7ZqSaYAAPNmJ?format=jpg&name=small
No Link Provided
No Link Provided
Attempting to access URL: https://pic.twitter.com/kadHdk8ML5
img_src: https://pbs.twimg.com/media/GOMP6tpboAAp_Re?format=jpg&name=small
Attempting to access URL: https://pic.twitter.com/kXdh92KP3p
img_src: https://pbs.twimg.com/media/GOMIUWKaMAAoILy?format=jpg&name=small
Attempting to access URL: https://pic.twitter.com/x4LiW2CwpL
img_src: https://pbs.twimg.com/media/GOHCcHgW0AE_cDV?format=jpg&name=small
Attempting to access URL: https://pic.twitter.com/eraAKf9z3s
img_src: https://pbs.

In [29]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(heisenberg_tweets)

,id,text,created_at,image_url,jpg_url
0,1795849187344699468,RIP.\n\nNeed a miracle to breakeven on this one.,2024-05-29 16:06:26+00:00,0,None
1,1795812171722723428,"$VIX Closed half, 25 contracts\n\nIn 2.98 (May 16th)\n\nOut 3.35\n\n+12.5%\n\nHolding other 25 contracts letting it ride a bit more. https://t.co/MTwtai7VTt",2024-05-29 13:39:20+00:00,[pic.twitter.com/MTwtai7VTt],https://pbs.twimg.com/media/GOwBptwacAAuYLR?format=jpg&name=small
2,1795555649134215214,$IWM well today’s entry was not ideal.\n\nBut this play was always going to be off Fri’s PCE data.\n\nSo HODL’ing.,2024-05-28 20:40:01+00:00,0,None
3,1795472181117551039,$IWM Open\n\nJust a small $1.9k bet for this Friday.\n\nWould LOVE to see a bagger on this one. https://t.co/szyGLaP9Fl,2024-05-28 15:08:20+00:00,[pic.twitter.com/szyGLaP9Fl],https://pbs.twimg.com/media/GOrMbrAbMAA-o5k?format=jpg&name=small
4,1795170192571384017,"$IWM Watchlist.\n\nIf inflation PCE #’s come in light, small caps should rip.\n\nProbably will keep my VIX play at bay if so. But I guess we can be a short term bull and a mid term mini bear.",2024-05-27 19:08:21+00:00,0,None
5,1795168704562385037,Earnings play this week:\n\n$ULTA\n\nGoing long in this one tomorrow to end of week. Hoping for a pre er run up.,2024-05-27 19:02:26+00:00,0,None
6,1793694236430434395,$VIX Pressed another 5 contracts\n\nTotal 50 contracts @ 2.98 avg.\n\n89 days for it to do something.\n\nEveryone is calling for a boring dull summer where VIX just dies.\n\nI went play contrarian and see that the VIX will have one more jump in it to 16-18’s within next 2-3 months.… https://t.co/BGSDDAGHpT https://t.co/dslHjMOuNR,2024-05-23 17:23:25+00:00,"[x.com/i/web/status/1…, pic.twitter.com/dslHjMOuNR]",https://pbs.twimg.com/media/GOR7ZqSaYAAPNmJ?format=jpg&name=small
7,1793378137189065025,Ftw!,2024-05-22 20:27:21+00:00,0,None
8,1793341078600577524,"A part of me really really feels the $VIX is due for a nice little pop any time now.\n\nThat does not mean the $SPX comes crashing, no.\n\nCan easily see a 20-30% VIX pop and only a SPX 1-1.5% drop.",2024-05-22 18:00:06+00:00,0,None
9,1793294582316151110,"$NVDA Open\n\nER play for today\n\nRisk $2,050 to make $450 https://t.co/kadHdk8ML5",2024-05-22 14:55:20+00:00,[pic.twitter.com/kadHdk8ML5],https://pbs.twimg.com/media/GOMP6tpboAAp_Re?format=jpg&name=small


In [38]:
def contains_twitter_pic(url_list):
    if not url_list:
        return False
    return any('pic.twitter' in url for url in url_list)

,id,text,created_at,image_url,jpg_url
0,1795472181117551039,$IWM Open\n\nJust a small $1.9k bet for this Friday.\n\nWould LOVE to see a bagger on this one. https://t.co/szyGLaP9Fl,2024-05-28 15:08:20+00:00,[pic.twitter.com/szyGLaP9Fl],https://pbs.twimg.com/media/GOrMbrAbMAA-o5k?format=jpg&name=small
1,1795170192571384017,"$IWM Watchlist.\n\nIf inflation PCE #’s come in light, small caps should rip.\n\nProbably will keep my VIX play at bay if so. But I guess we can be a short term bull and a mid term mini bear.",2024-05-27 19:08:21+00:00,0,None
2,1795168704562385037,Earnings play this week:\n\n$ULTA\n\nGoing long in this one tomorrow to end of week. Hoping for a pre er run up.,2024-05-27 19:02:26+00:00,0,None
3,1793694236430434395,$VIX Pressed another 5 contracts\n\nTotal 50 contracts @ 2.98 avg.\n\n89 days for it to do something.\n\nEveryone is calling for a boring dull summer where VIX just dies.\n\nI went play contrarian and see that the VIX will have one more jump in it to 16-18’s within next 2-3 months.… https://t.co/BGSDDAGHpT https://t.co/dslHjMOuNR,2024-05-23 17:23:25+00:00,"[x.com/i/web/status/1…, pic.twitter.com/dslHjMOuNR]",https://pbs.twimg.com/media/GOR7ZqSaYAAPNmJ?format=jpg&name=small
4,1793378137189065025,Ftw!,2024-05-22 20:27:21+00:00,0,None
5,1793341078600577524,"A part of me really really feels the $VIX is due for a nice little pop any time now.\n\nThat does not mean the $SPX comes crashing, no.\n\nCan easily see a 20-30% VIX pop and only a SPX 1-1.5% drop.",2024-05-22 18:00:06+00:00,0,None
6,1793294582316151110,"$NVDA Open\n\nER play for today\n\nRisk $2,050 to make $450 https://t.co/kadHdk8ML5",2024-05-22 14:55:20+00:00,[pic.twitter.com/kadHdk8ML5],https://pbs.twimg.com/media/GOMP6tpboAAp_Re?format=jpg&name=small
7,1793286225878761645,$PZZA Open\n\nTrying to catch a falling knife here. https://t.co/kXdh92KP3p,2024-05-22 14:22:08+00:00,[pic.twitter.com/kXdh92KP3p],https://pbs.twimg.com/media/GOMIUWKaMAAoILy?format=jpg&name=small
8,1792927921704710336,$WBA Open\n\nAnother contrarian play.\n\n241 days to do something.\n\nA move back to $20 should get a bagger on these near leaps. https://t.co/x4LiW2CwpL,2024-05-21 14:38:22+00:00,[pic.twitter.com/x4LiW2CwpL],https://pbs.twimg.com/media/GOHCcHgW0AE_cDV?format=jpg&name=small
9,1792926733584211971,$PTON Open\n\nThis is what I call a “why not” play.\n\nBinary. Either this goes to $0-1 or it goes to $7-10.\n\nAll hinders on a takeover imo.\n\nGot 241 days to do something. https://t.co/eraAKf9z3s,2024-05-21 14:33:38+00:00,[pic.twitter.com/eraAKf9z3s],https://pbs.twimg.com/media/GOHBW5ZXgAENuEg?format=jpg&name=small


In [72]:
heisenberg_tweets.loc[:,'image_response'] = heisenberg_tweets['jpg_url'].apply(lambda x: get_response_image(x) if x != None else None)
heisenberg_tweets['image_response'] = heisenberg_tweets['image_response'].apply(lambda x: x.replace('$', '\$') if x and pd.notna(x) else x)
heisenberg_tweets['text'] = heisenberg_tweets['text'].apply(lambda x: x.replace('$', '\$') if pd.notna(x) else x)
heisenberg_tweets['full_response'] = heisenberg_tweets['text'].astype(str) + '  TRANSCRIBED IMAGE DATA: ' + heisenberg_tweets['image_response'].astype(str)

cwd = os.getcwd()
file_path = os.path.join(cwd, 'ht_unprocessed.xlsx')
heisenberg_tweets['created_at'] = heisenberg_tweets['created_at'].dt.tz_localize(None)
heisenberg_tweets = heisenberg_tweets.map(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
print(file_path)
heisenberg_tweets.to_excel(file_path, index=False)

<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:3: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:3: SyntaxWarning: invalid escape sequence '\$'
C:\Users\abdan\AppData\Local\Temp\ipykernel_15796\2963335271.py:2: SyntaxWarning: invalid escape sequence '\$'
  heisenberg_tweets['image_response'] = heisenberg_tweets['image_response'].apply(lambda x: x.replace('$', '\$') if x and pd.notna(x) else x)
C:\Users\abdan\AppData\Local\Temp\ipykernel_15796\2963335271.py:3: SyntaxWarning: invalid escape sequence '\$'
  heisenberg_tweets['text'] = heisenberg_tweets['text'].apply(lambda x: x.replace('$', '\$') if pd.notna(x) else x)


C:\Users\abdan\Project_X\ht_unprocessed.xlsx


In [73]:
heisenberg_tweets.head(10)

,id,text,created_at,image_url,jpg_url,image_response,full_response
0,1795170192571384017,"\\$IWM Watchlist.\n\nIf inflation PCE #\u2019s come in light, small caps should rip.\n\nProbably will keep my VIX play at bay if so. But I guess we can be a short term bull and a mid term mini bear.",2024-05-27 19:08:21,0,None,None,"\\$IWM Watchlist.\n\nIf inflation PCE #\u2019s come in light, small caps should rip.\n\nProbably will keep my VIX play at bay if so. But I guess we can be a short term bull and a mid term mini bear. TRANSCRIBED IMAGE DATA: None"
1,1795168704562385037,Earnings play this week:\n\n\\$ULTA\n\nGoing long in this one tomorrow to end of week. Hoping for a pre er run up.,2024-05-27 19:02:26,0,None,None,Earnings play this week:\n\n\\$ULTA\n\nGoing long in this one tomorrow to end of week. Hoping for a pre er run up. TRANSCRIBED IMAGE DATA: None
2,1793694236430434395,\\$VIX Pressed another 5 contracts\n\nTotal 50 contracts @ 2.98 avg.\n\n89 days for it to do something.\n\nEveryone is calling for a boring dull summer where VIX just dies.\n\nI went play contrarian and see that the VIX will have one more jump in it to 16-18\u2019s within next 2-3 months.\u2026 https://t.co/BGSDDAGHpT https://t.co/dslHjMOuNR,2024-05-23 17:23:25,"[x.com/i/web/status/1…, pic.twitter.com/dslHjMOuNR]",https://pbs.twimg.com/media/GOR7ZqSaYAAPNmJ?format=jpg&name=small,"This image describes the purchase of a call option. Specifically, it is a call option for the VIX (Volatility Index) with an expiration date of August 21, 2024, and a strike price of 12.5. The trade is for 5 contracts at a price of 2.9 per contract.","\\$VIX Pressed another 5 contracts\n\nTotal 50 contracts @ 2.98 avg.\n\n89 days for it to do something.\n\nEveryone is calling for a boring dull summer where VIX just dies.\n\nI went play contrarian and see that the VIX will have one more jump in it to 16-18\u2019s within next 2-3 months.\u2026 https://t.co/BGSDDAGHpT https://t.co/dslHjMOuNR TRANSCRIBED IMAGE DATA: This image describes the purchase of a call option. Specifically, it is a call option for the VIX (Volatility Index) with an expiration date of August 21, 2024, and a strike price of 12.5. The trade is for 5 contracts at a price of 2.9 per contract."
3,1793378137189065025,Ftw!,2024-05-22 20:27:21,0,None,None,Ftw! TRANSCRIBED IMAGE DATA: None
4,1793341078600577524,"A part of me really really feels the \\$VIX is due for a nice little pop any time now.\n\nThat does not mean the \\$SPX comes crashing, no.\n\nCan easily see a 20-30% VIX pop and only a SPX 1-1.5% drop.",2024-05-22 18:00:06,0,None,None,"A part of me really really feels the \\$VIX is due for a nice little pop any time now.\n\nThat does not mean the \\$SPX comes crashing, no.\n\nCan easily see a 20-30% VIX pop and only a SPX 1-1.5% drop. TRANSCRIBED IMAGE DATA: None"
5,1793294582316151110,"\\$NVDA Open\n\nER play for today\n\nRisk \\$2,050 to make \\$450 https://t.co/kadHdk8ML5",2024-05-22 14:55:20,[pic.twitter.com/kadHdk8ML5],https://pbs.twimg.com/media/GOMP6tpboAAp_Re?format=jpg&name=small,"This image describes an options trading strategy known as a ""vertical spread,"" specifically a ""vertical put spread.""\n\nHere are the details:\n\n- **Stock**: NVDA (NVIDIA Corporation)\n- **Expiration Date**: 24 May 2024\n- **Option Type**: Put\n- **Strike Prices**: 860/855\n- **Quantity**: 5 contracts\n\nIn this case, it's a vertical put spread, also referred to as a bull put spread:\n\n1. **Sell to open 5 contracts of the NVDA 860 Puts**\n2. **Buy to open 5 contracts of the NVDA 855 Puts**\n\nThe net credit received for this spread is 0.90 (as indicated by the ""CREDIT"" notation), meaning the trader received \\$0.90 per share for a total of \\$90 for each spread (since each options contract represents 100 shares).\n\nThis strategy profits if NVDA stock price stays above the higher strike price (860) by expiration. The maximum gain is the net credit received, and the maximum loss is the difference between the strike prices

In [74]:
#I've separated the prompting into two separate columns, one to get the ticker (here) and another to get the direction (below)

cols = ['id', 'created_at', 'full_response']
ht_dynamic = heisenberg_tweets[cols]
sys_prompt = """You are parsing tweets to interpret and synthesize information about stock plays. Reference examples as a guide to understand the format of the output.
Example:
Text: $PARA Closed\n\nIn 13.11 (yesterday)\n\nOut 13.24\n\n+1%\n+$65 profit\n\nJust trying to reduce long exposure heading into tomorrow. 
https://t.co/GpCKwDrfky 
TRANSCRIBED IMAGE DATA: This image describes a stock sale transaction, not an option play like a call or a put. 
Specifically, it details the sale of 500 shares of PARA (which is the ticker symbol for a stock) at an average fill price of $13.2401. 
It's a limit order set to sell at $13.24. The negative quantity (-500) indicates that shares are being sold rather than purchased.
Correct Output: [Close PARA Long]

Text: If $META closes above 450, I will do 1,000 jumping jacks.\n\n🙏
TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]

Text: \\$GOOG Closed\n\nIn .32 credit (Apr 30th)\n\nOut .05 debit\n\n+\\$135 profit\n\nSmall. Got dangerously close. But never worried. TRANSCRIBED IMAGE DATA: The image describes a vertical options spread for Google (GOOG) stock, specifically a put spread. Here are the details:\n\n- **Type of Spread**: Vertical Put Spread\n- **Underlying Asset**: GOOG (Google)\n- **Expiry Date**: May 3, 2024\n- **Strike Prices**: 165 (bought) and 162.5 (sold)\n\nDetails of the trade:\n- **Quantity**: 5 contracts\n- **Total Cost**: Since you are paying 0.05 per contract for 5 contracts, the total debit is 5 * 0.05 = \\$25.\n\nIn this trade, you are buying 5 put options at a strike price of 165 and selling 5 put options at a strike price of 162.5 for the same expiration date of May 3, 2024. This creates a vertical spread (specifically a bear put spread), aiming to profit from a decrease in the price of the underlying stock, Google, down to or below the lower strike price of 162.5.
Correct Output: [Close GOOG Short]

Text: \\$AMD Eyeing this name.\n\nIf it can get to 137-138ish tomorrow or next week, then I would buy there. \n\nWhy? 200dma magnet could act as major support.  TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]

Text: \\$DIS Open\n\nEarnings play\n\nRisk \\$1,670 to make \\$330.\n\nBmo, implied about a 7pt move. https://t.co/nyMZGS23OZ  TRANSCRIBED IMAGE DATA: The image describes a vertical put spread (also known as a bull put spread). This is a type of options strategy that involves selling one put option and buying another put option at a lower strike price but with the same expiration date.\n\nHere's the breakdown of the trade based on the image:\n\n- This is for the stock with the ticker symbol "DIS" (Walt Disney Company), with options expiring on May 10, 2024.\n- The vertical spread involves the 108 and 106 strike prices, meaning you are dealing with 108/106 put options.\n- The strategy specified is a put vertical spread:\n  - Selling 10 put options at the 108 strike price.\n  - Buying 10 put options at the 106 strike price.\n- The prices for the transactions are:\n  - Sold (shorted) the 108 strike put options at \\$0.86 each.\n  - Bought (long) the 106 strike put options at \\$0.53 each.\n- The net credit received for the spread is \\$0.33 per share (since options typically represent 100 shares, the total net credit is \\$33 per contract).\n\nIn summary, the strategy is a bull put spread where you hope the price of Disney (DIS) stays above the higher strike price (108) by the expiration date so that both options expire worthless, and you keep the credit received upfront.
Correct Output: [Open DIS Long]

Text: Watchlist for next week:\n\nLong: \\$VIX\n\nShort: \\$FXI, \\$SPY\n\nNeutral: \\$NVDA\n\nSpeculative bounce play on watch: \\$SHOP  TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]

Text: Two Biggest Losers this week:\n\nSPY call credit spread: -\\$5,130\n\nDIS put credit spread: -\\$1,670\n\nOuch.\n\nAccount Balance: \\$114,888.39.\n\n-\\$5,097.20 week over week.\n\nNot happy, but we look forward to next week. https://t.co/X67fSZOUY6  TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]

Text: \\\\$FXI Open\\n\\nJust some yolo puts here looking for a healthy small pullback.\\n\\nIndex up 10 of last 11 days. https://t.co/PWuTtGhm5t  TRANSCRIBED IMAGE DATA: The image describes the purchase of a put option. Specifically, the details are as follows:\\n\\n- Ticker Symbol: FXI 100 (This commonly refers to an exchange-traded fund (ETF) that tracks the performance of the top 100 Chinese companies).\\n- Weekly options expiring on: May 10th, 2024.\\n- Strike Price: 27.5.\\n- Type: Put option (designated by the "P").\\n- Quantity: 15 contracts.\\n- Price: \\\\$0.44 per contract (with a limit order as indicated by "LMT").\\n- Trade fill date and time: May 6th, 2024, at 6:32 AM.\\n\\nThis indicates that the trader bought 15 put options at a strike price of 27.5, and they paid \\\\$0.44 per option contract.
Correct Output: [Open FXI Short]

Text: Hang Seng going bonkers overnight.\\nIf it sticks by US open, these FXI calls should be close to 100%!  TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]

Text: \\\\$GME selling call spreads on watch.  TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]

Text: For those who want to play the VIX but cannot because cough Robinhood cough, then I suggest SPY puts. It's pretty much the next best alternative.\\n\\nDon't give me that UVXY crap imo.  TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]
"""
user_prompt = "Is this tweet referring to the opening or closing of a stock position? If it is, please also list the corresponding ticker and whether it is long or short. If it is not referring to the opening or closing of a position, simply put neither. Please respond in the possible formats: [Open/Close TICKER Long/Short] or [Neither]. If the tweet refers to multiple positions, list them all in a comma separated list."
ht_dynamic['result'] = ht_dynamic['full_response'].apply(
    lambda text: dynamic_prompting(text, sys_prompt, user_prompt)
)


C:\Users\abdan\AppData\Local\Temp\ipykernel_15796\302103987.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ht_dynamic['result'] = ht_dynamic['full_response'].apply(


In [75]:
cwd = os.getcwd()
file_path = os.path.join(cwd, 'tweet_consolidated.xlsx')
ht_dynamic['created_at'] = ht_dynamic['created_at'].dt.tz_localize(None)
ht_dynamic = ht_dynamic.map(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
print(file_path)
ht_dynamic.to_excel(file_path, index=False)

C:\Users\abdan\Project_X\tweet_consolidated.xlsx


C:\Users\abdan\AppData\Local\Temp\ipykernel_15796\1194833683.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ht_dynamic['created_at'] = ht_dynamic['created_at'].dt.tz_localize(None)
